In [68]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

In [27]:
# Load the data from the CSV file
file_path = r'C:\Users\P56895\Master thesis\output.csv'  # Update the path accordingly
df = pd.read_csv(file_path)

# Convert 'time' column to datetime object
df['time'] = pd.to_datetime(df['time'])

# Drop 'longitude' and 'latitude' columns
df.drop(['longitude', 'latitude'], axis=1, inplace=True)


In [28]:
# Transform 'time' to integer
df['time_int'] = (df['time'] - df['time'].min()).dt.days.astype(int)

# Calculate the average of features over all locations for each timestep
avg_features = df.groupby('time').mean().reset_index()

# Replace NaN values with the mean value for each feature
avg_features = avg_features.apply(lambda col: col.fillna(col.mean()))


In [29]:

# Drop the original 'time' column
avg_features.drop('time', axis=1, inplace=True)
avg_features

,v10,t2m,niaod550,pm10,pm1,pm2p5,ssaod550,sp,tcco,tc_hno3,tcno2,u10,time_int
0,0.828576,282.111528,0.019976,9.609748e-09,3.915548e-09,6.044937e-09,0.024591,98771.488015,0.000835,0.000003,0.000006,4.724723,0.0
1,3.523364,279.978376,0.015238,1.277753e-08,6.202240e-09,8.303599e-09,0.007245,98642.561854,0.000733,0.000003,0.000007,1.662859,1.0
2,2.467317,281.405031,0.015428,8.433507e-09,3.155755e-09,5.179849e-09,0.031107,97741.780759,0.000836,0.000004,0.000006,5.966085,2.0
3,2.988695,279.905962,0.011879,7.512874e-09,2.700119e-09,4.587937e-09,0.027055,97025.993396,0.000802,0.000004,0.000006,3.676209,3.0
4,1.357448,276.540026,0.006968,7.756444e-09,2.041516e-09,4.525248e-09,0.028665,96407.610278,0.000820,0.000004,0.000008,4.192985,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,2.284676,287.543963,0.039634,1.745904e-08,9.253056e-09,1.167091e-08,0.012987,98480.435807,0.000927,0.000011,0.000008,0.075234,636.0
637,0.633387,281.897038,0.024856,1.580960e-08,6.747306e-09,1.009696e-08,0.047065,98321.190954,0.001140,0.000012,0.000008,0.817964,637.0
638,0.633387,281.897038,0.027791,2.064923e-08,1.020907e-08,1.356638e-08,0.037740,98321.190954,0.000973,0.000011,0.000008,0.817964,638.0
639,0.633387,281.897038,0.032038,1.573288e-08,8.353854e-09,1.058978e-08,0.012692,98321.190954,0.000846,0.000012,0.000008,0.817964,639.0


In [51]:

# Define the number of lag days
n_lags = 30  # Adjust based on your analysis

# Create lagged features for all variables, including the target
for col in avg_features.columns:
    if col != 'time_int':
        for i in range(1, n_lags + 1):
            avg_features[f'{col}_lag_{i}'] = avg_features[col].shift(i)


C:\Users\P56895\.conda\envs\geo-netcdf\lib\site-packages\ipykernel_launcher.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [52]:
# Replace NaN values with the mean value for each feature
avg_features = avg_features.apply(lambda col: col.fillna(col.mean())) # if we do not apply this, the first lagged features will be replaced with NaN values
# Sort features and targets based on 'time_int'
avg_features.reset_index(drop=True, inplace=True)

avg_features['pm2p5_lag_14']

0      9.449232e-09
1      9.449232e-09
2      9.449232e-09
3      9.449232e-09
4      9.449232e-09
           ...     
636    8.622960e-09
637    8.553200e-09
638    1.105205e-08
639    1.014802e-08
640    8.189499e-09
Name: pm2p5_lag_14, Length: 641, dtype: float64

In [53]:
# Separate features and target variables
features = avg_features.drop(['niaod550', 'pm10', 'pm1', 'pm2p5', 'ssaod550','tc_hno3','tcco', 'tcno2'], axis=1)
targets = avg_features[['niaod550', 'pm10', 'pm1', 'pm2p5', 'ssaod550','tc_hno3','tcco', 'tcno2']]


In [54]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [55]:
# Initialize a dictionary to store models for each target
gb_regressors = {}

# Train one model for each target variable
for target in targets.columns:
    # Initialize the Gradient Boosting Regressor
    gb_regressor = GradientBoostingRegressor(n_estimators=100, random_state=42)

    # Train the model for the current target variable
    gb_regressor.fit(X_train_scaled, y_train[target])

    # Store the trained model in the dictionary
    gb_regressors[target] = gb_regressor

    # Make predictions on the test set
    predictions = gb_regressor.predict(X_test_scaled)

    # Evaluate the model
    mse = mean_squared_error(y_test[target], predictions)
    print(f'Mean Squared Error for {target}: {mse}')

Mean Squared Error for niaod550: 0.00013889207206034242
Mean Squared Error for pm10: 2.2990832634197338e-17
Mean Squared Error for pm1: 8.50158602366734e-18
Mean Squared Error for pm2p5: 1.1032958884335573e-17
Mean Squared Error for ssaod550: 6.615862325170335e-05
Mean Squared Error for tcco: 3.2489964728304156e-09
Mean Squared Error for tcno2: 4.1714604313725065e-13


In [56]:
gb_regressors

{'niaod550': GradientBoostingRegressor(random_state=42),
 'pm10': GradientBoostingRegressor(random_state=42),
 'pm1': GradientBoostingRegressor(random_state=42),
 'pm2p5': GradientBoostingRegressor(random_state=42),
 'ssaod550': GradientBoostingRegressor(random_state=42),
 'tcco': GradientBoostingRegressor(random_state=42),
 'tcno2': GradientBoostingRegressor(random_state=42)}

In [71]:
# Number of days to forecast
forecast_days = 30

# Identify the last timestamp in the available data
last_timestamp = avg_features['time_int'].max()

# Select the last row of the average features data for the identified timestamp
last_data = avg_features[avg_features['time_int'] == last_timestamp][features.columns].values

# Standardize the features for the last data point
last_data_scaled = scaler.transform(last_data)


C:\Users\P56895\.conda\envs\geo-netcdf\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


In [74]:
# Initialize an array to store forecasts
forecasts = np.zeros((forecast_days, targets.shape[1]))


In [82]:
# Iterate over the forecast days
for day in range(forecast_days):
    forecast_values = []

    # Iterate over each target variable
    for target in targets.columns:
        # Use the corresponding model from the dictionary
        model = gb_regressors[target]

        # Make a prediction for the next day
        forecast = model.predict(last_data)

        # Append the forecasted value
        forecast_values.append(forecast[0])

        # Update lagged features for the next iteration
        last_data[:, 1:] = np.roll(last_data[:, 1:], shift=1, axis=1)
        last_data[:, 0] = forecast

    # Store the forecast values for the current day
    forecasts[day, :] = forecast_values

In [86]:
# Extract the last timestamp from the DataFrame
last_timestamp = df['time'].max()

# Generate the right timestamps for the forecasts
forecast_timestamps = pd.date_range(start=last_timestamp + pd.Timedelta(days=1), periods=forecast_days, freq='D')

# Create a DataFrame for the forecasts
forecasts_df = pd.DataFrame(forecasts, index=forecast_timestamps, columns=targets.columns)

# Print the forecasts DataFrame
print(forecasts_df)

            niaod550          pm10           pm1         pm2p5  ssaod550  \
2023-10-04  0.041332  1.457976e-08  7.046855e-09  9.534282e-09  0.009232   
2023-10-05  0.027585  1.457976e-08  7.046855e-09  9.534282e-09  0.008382   
2023-10-06  0.034871  1.457976e-08  7.046855e-09  9.534282e-09  0.002245   
2023-10-07  0.050422  1.457976e-08  7.046855e-09  9.534282e-09  0.008742   
2023-10-08  0.031965  1.457976e-08  7.046855e-09  9.534282e-09  0.016274   
2023-10-09  0.033294  1.457976e-08  7.046855e-09  9.534282e-09  0.011690   
2023-10-10  0.032953  1.457976e-08  7.046855e-09  9.534282e-09  0.001709   
2023-10-11  0.040173  1.457976e-08  7.046855e-09  9.534282e-09  0.016072   
2023-10-12  0.036387  1.457976e-08  7.046855e-09  9.534282e-09  0.011878   
2023-10-13  0.034893  1.457976e-08  7.046855e-09  9.534282e-09  0.000345   
2023-10-14  0.031931  1.457976e-08  7.046855e-09  9.534282e-09  0.001407   
2023-10-15  0.034296  1.457976e-08  7.046855e-09  9.534282e-09  0.018297   
2023-10-16  

In [91]:
# Save the forecasts DataFrame to a CSV file with the 'time' column name
forecasts_df.to_csv('forecasts.csv', index_label='time')

# Optionally, you can specify the path where you want to save the file:
# forecasts_df.to_csv('/path/to